### Beagle

In [1]:
%%bash
set -euo pipefail

# Input
MASKED_VCF="/mnt/qmtang/EvoFill_data/20251211_chr22/data/minor_pops_all.mask90p.vcf.gz" 
REF_VCF="/mnt/qmtang/EvoFill_data/20251211_chr22/data/major_pops_train.vcf.gz"
# Output
OUT_DIR="/home/zqyin/mamba_test/Other_Model_test/chr22/Beagle"
OUT_PREFIX="${OUT_DIR}/minor_pops_all.mask90p.imputed.Beagle"
mkdir -p "${OUT_DIR}"

# Time Tracking log
TIME_LOG="${OUT_PREFIX}.time.log"

echo "[INFO] Start Beagle: $(date)"

# Impution
/usr/bin/time -v \
  beagle \
    -Xms4g -Xmx64g \
    gt=${MASKED_VCF} \
    ref=${REF_VCF} \
    out=${OUT_PREFIX} \
    impute=true \
    gp=true \
    ap=true \
    nthreads=8 \
  2> "${TIME_LOG}"

echo "[INFO] End Beagle: $(date)"
echo "[INFO] Time/Memory stats saved in: ${TIME_LOG}"


[INFO] Start Beagle: Mon Jan 12 09:45:33 AM CST 2026
beagle.27Feb25.75f.jar (version 5.5)
Copyright (C) 2014-2024 Brian L. Browning
Enter "java -jar beagle.27Feb25.75f.jar" to list command line argument
Start time: 01:45 AM UTC on 12 Jan 2026

Command line: java -Xmx65536m -jar beagle.27Feb25.75f.jar
  gt=/mnt/qmtang/EvoFill_data/20251211_chr22/data/minor_pops_all.mask90p.vcf.gz
  ref=/mnt/qmtang/EvoFill_data/20251211_chr22/data/major_pops_train.vcf.gz
  out=/home/zqyin/mamba_test/Other_Model_test/chr22/Beagle/minor_pops_all.mask90p.imputed.Beagle
  impute=true
  gp=true
  ap=true
  nthreads=8

No genetic map is specified: using 1 cM = 1 Mb

Reference samples:                3,009
Study     samples:                   93

Window 1 [chr22:10519276-50519196]
Reference markers:              188,602
Study     markers:              188,602

Burnin  iteration 1:           42 seconds
Burnin  iteration 2:           19 seconds
Burnin  iteration 3:           19 seconds

Estimated ne:             

### Minimac

In [2]:
%%bash
#!/usr/bin/env bash
set -e
set -u
(set -o pipefail) 2>/dev/null || true

# Input
MASKED_VCF="/mnt/qmtang/EvoFill_data/20251211_chr22/data/minor_pops_all.mask90p.vcf.gz"
REF_VCF="/mnt/qmtang/EvoFill_data/20251211_chr22/data/major_pops_train.vcf.gz"

# Output
OUT_DIR="/home/zqyin/mamba_test/Other_Model_test/chr22/Minimac"
mkdir -p "${OUT_DIR}"

THREADS=8
REGION="chr22"

MSAV_REF="${OUT_DIR}/major_pops_chr22.msav"
IMPUTED_VCF="${OUT_DIR}/minor_pops_all.mask90p.imputed.minimac4.vcf.gz"
IMPUTED_SAV="${OUT_DIR}/minor_pops_all.mask90p.imputed.minimac4.sav"
SITES_VCF="${OUT_DIR}/minor_pops_all.mask90p.imputed.minimac4.sites.vcf.gz"

# Only count the time for the Minimac step
TIME_LOG="${OUT_DIR}/minor_pops_all.mask90p.imputed.minimac4.time.log"

IMPUTED_VCF_NOPROB="${OUT_DIR}/minor_pops_all.mask90p.imputed.minimac4.noGP.vcf.gz"

echo "[INFO] minimac4 Path: $(command -v minimac4)"

echo "[CHECK] Checking whether REF_VCF is phased..."
PHASED_REF=$(bcftools query -f '[%GT]\n' "${REF_VCF}" | head -n 100000 | grep -c '|') || true
if [[ "${PHASED_REF}" -eq 0 ]]; then
  echo "[ERROR] Reference panel ${REF_VCF} is not phased (no '|' detected). Please phase it first." >&2
  exit 1
fi

echo "[CHECK] Checking whether MASKED_VCF is phased..."
PHASED_TGT=$(bcftools query -f '[%GT]\n' "${MASKED_VCF}" | head -n 100000 | grep -c '|') || true
if [[ "${PHASED_TGT}" -eq 0 ]]; then
  echo "[ERROR] Target file ${MASKED_VCF} is not phased (no '|' detected). Please phase it first." >&2
  exit 1
fi

# ======== Indexing (not counted toward Minimac time) ========
if [[ ! -f "${REF_VCF}.tbi" && ! -f "${REF_VCF}.csi" ]]; then
  echo "[INFO] Building index for REF_VCF..."
  tabix -p vcf "${REF_VCF}"
fi

if [[ ! -f "${MASKED_VCF}.tbi" && ! -f "${MASKED_VCF}.csi" ]]; then
  echo "[INFO] Building index for MASKED_VCF..."
  tabix -p vcf "${MASKED_VCF}"
fi

# ======== Step 1: Generate msav (not counted toward Minimac time) ========
echo "[STEP 1/4] Generating reference panel: ${MSAV_REF}"
minimac4 --compress-reference "${REF_VCF}" > "${MSAV_REF}"
echo "[OK] msav reference panel completed"

# ======== Step 2: Time only the Minimac imputation ========
echo "[STEP 2/4] Running Minimac4 (region: ${REGION})..."
echo "[INFO] Start Minimac4: $(date)"

 /usr/bin/time -v \
  minimac4 \
    "${MSAV_REF}" \
    "${MASKED_VCF}" \
    --region "${REGION}" \
    --output "${IMPUTED_VCF}" \
    --empirical-output "${IMPUTED_SAV}" \
    --sites "${SITES_VCF}" \
    --format GT,DS,GP,HDS \
    --threads "${THREADS}" \
  2> "${TIME_LOG}"

echo "[INFO] End Minimac4: $(date)"
echo "[INFO] Minimac4 timing statistics saved to: ${TIME_LOG}"

tabix -p vcf "${IMPUTED_VCF}"
tabix -p vcf "${SITES_VCF}"

echo "[OK] Imputation completed:"
echo "  - VCF  : ${IMPUTED_VCF}"
echo "  - SITES: ${SITES_VCF}"
echo "  - SAV  : ${IMPUTED_SAV}"

# ======== Step 3: Generate VCF without probability fields ========
echo "[STEP 3/4] Generating VCF with probability fields (DS, GP, HDS) removed..."
bcftools annotate \
  -x FORMAT/DS,FORMAT/GP,FORMAT/HDS \
  -Oz -o "${IMPUTED_VCF_NOPROB}" \
  "${IMPUTED_VCF}"

tabix -p vcf "${IMPUTED_VCF_NOPROB}"

echo "[OK] VCF without probability fields: ${IMPUTED_VCF_NOPROB}"

# ======== Step 4: Missing rate comparison (not counted toward Minimac time) ========
echo "[STEP 4/4] Rough missing rate statistics (before vs after imputation)"

echo -n "  [Before] MASKED_VCF missing rate: "
bcftools query -f '[%GT]\n' "${MASKED_VCF}" | tr '|' '/' | \
awk 'BEGIN{FS="[ /]"}{for(i=1;i<=NF;i++) if($i==".") m++; t+=NF} END{if(t>0) printf("%.4f\n", m/t); else print "NA"}'

echo -n "  [After ] IMPUTED_VCF missing rate: "
bcftools query -f '[%GT]\n' "${IMPUTED_VCF}" | tr '|' '/' | \
awk 'BEGIN{FS="[ /]"}{for(i=1;i<=NF;i++) if($i==".") m++; t+=NF} END{if(t>0) printf("%.4f\n", m/t); else print "NA"}'

echo "[ALL DONE] Output directory: ${OUT_DIR}"
echo "  - VCF with probabilities    : ${IMPUTED_VCF}"
echo "  - VCF without probabilities: ${IMPUTED_VCF_NOPROB}"


[INFO] minimac4 Path: /home/zqyin/minimac4-4.1.6/minimac4-4.1.6-Linux-x86_64/bin/minimac4
[CHECK] Checking whether REF_VCF is phased...
[CHECK] Checking whether MASKED_VCF is phased...
[STEP 1/4] Generating reference panel: /home/zqyin/mamba_test/Other_Model_test/chr22/Minimac/major_pops_chr22.msav


minimac v4.1.6

Mean Compression Ratio: 0.0606707
Min Compression Ratio: 0.0187287
Max Compression Ratio: 0.265636


[OK] msav reference panel completed
[STEP 2/4] Running Minimac4 (region: chr22)...
[INFO] Start Minimac4: Mon Jan 12 10:07:02 AM CST 2026
[INFO] End Minimac4: Mon Jan 12 10:10:03 AM CST 2026
[INFO] Minimac4 timing statistics saved to: /home/zqyin/mamba_test/Other_Model_test/chr22/Minimac/minor_pops_all.mask90p.imputed.minimac4.time.log
[OK] Imputation completed:
  - VCF  : /home/zqyin/mamba_test/Other_Model_test/chr22/Minimac/minor_pops_all.mask90p.imputed.minimac4.vcf.gz
  - SITES: /home/zqyin/mamba_test/Other_Model_test/chr22/Minimac/minor_pops_all.mask90p.imputed.minimac4.sites.vcf.gz
  - SAV  : /home/zqyin/mamba_test/Other_Model_test/chr22/Minimac/minor_pops_all.mask90p.imputed.minimac4.sav
[STEP 3/4] Generating VCF with probability fields (DS, GP, HDS) removed...
[OK] VCF without probability fields: /home/zqyin/mamba_test/Other_Model_test/chr22/Minimac/minor_pops_all.mask90p.imputed.minimac4.noGP.vcf.gz
[STEP 4/4] Rough missing rate statistics (before vs after imputation)
  [Befor

### Eagle

In [1]:
%%bash
#!/usr/bin/env bash
set -euo pipefail

# =========================
# 0) Input / Output (as provided)
# =========================
MASKED_VCF="/mnt/qmtang/EvoFill_data/20251211_chr22/data/minor_pops_all.mask90p.vcf.gz"
REF_VCF="/mnt/qmtang/EvoFill_data/20251211_chr22/data/major_pops_train.vcf.gz"

OUT_DIR="/home/zqyin/mamba_test/Other_Model_test/chr22/Eagle"
OUT_PREFIX="${OUT_DIR}/minor_pops_all.mask90p.imputed.Eagle"

# Eagle official hg38 genetic map
MAP="/home/zqyin/mamba_test/Other_Model_test/chr22/GLIMPSE2/genetic_map_hg38_withX.txt.gz"

THREADS=8
CHR="chr22"

# =========================
# 1) Basic checks
# =========================
mkdir -p "${OUT_DIR}"/{tmp,logs}

need_cmd() {
  command -v "$1" >/dev/null 2>&1 || {
    echo "[ERROR] missing command: $1" >&2
    exit 1
  }
}
need_cmd eagle
need_cmd bcftools
need_cmd /usr/bin/time

for f in "$MASKED_VCF" "$REF_VCF" "$MAP"; do
  [[ -e "$f" ]] || { echo "[ERROR] file not found: $f" >&2; exit 1; }
done

echo "[INFO] MASKED_VCF = $MASKED_VCF"
echo "[INFO] REF_VCF    = $REF_VCF"
echo "[INFO] MAP        = $MAP"
echo "[INFO] OUT_PREFIX = $OUT_PREFIX"
echo "[INFO] THREADS    = $THREADS"
echo "[INFO] CHR        = $CHR"

# =========================
# 2) Ensure input VCFs are indexed
# =========================
echo "[INFO] Indexing input VCFs if needed..."
bcftools index -t "$MASKED_VCF" 2>/dev/null || true
bcftools index -t "$REF_VCF"    2>/dev/null || true

# =========================
# 3) Convert to BCF (recommended for speed) and index
# =========================
TMP="${OUT_DIR}/tmp"
TGT_BCF="${TMP}/target.${CHR}.bcf"
REF_BCF="${TMP}/ref.${CHR}.bcf"

echo "[INFO] Converting VCF to BCF..."
bcftools view -Ob -o "$TGT_BCF" "$MASKED_VCF"
bcftools index -f "$TGT_BCF"

bcftools view -Ob -o "$REF_BCF" "$REF_VCF"
bcftools index -f "$REF_BCF"

# =========================
# 4) Run Eagle (reference-based phasing + GT imputation)
#
# --outputUnphased     : keep target-only sites (not present in reference)
# --allowRefAltSwap    : automatically fix REF/ALT swaps to reduce site loss
#
# Timing is applied ONLY to Eagle itself
# =========================
LOG="${OUT_DIR}/logs/eagle.${CHR}.$(date +%Y%m%d-%H%M%S).log"
TIME_LOG="${OUT_DIR}/logs/eagle.${CHR}.$(date +%Y%m%d-%H%M%S).time"

echo "[INFO] Running Eagle..."
echo "[INFO] Start Eagle: $(date)"

set -x
/usr/bin/time -v \
  eagle \
    --vcfTarget="$TGT_BCF" \
    --vcfRef="$REF_BCF" \
    --geneticMapFile="$MAP" \
    --chrom="$CHR" \
    --outPrefix="$OUT_PREFIX" \
    --vcfOutFormat=z \
    --numThreads="$THREADS" \
    --outputUnphased \
    --allowRefAltSwap \
  2> "$TIME_LOG" | tee "$LOG"
set +x

echo "[INFO] End Eagle: $(date)"
echo "[INFO] Eagle timing statistics saved to: $TIME_LOG"

# =========================
# 5) Output indexing and simple QC
# =========================
OUT_VCF="${OUT_PREFIX}.vcf.gz"
echo "[INFO] Output VCF: ${OUT_VCF}"

bcftools index -t "$OUT_VCF"

echo "[INFO] QC: samples and sites"
echo -n "  #samples = "
bcftools query -l "$OUT_VCF" | wc -l
echo -n "  #sites   = "
bcftools view -H "$OUT_VCF" | wc -l

echo "[DONE] Eagle finished."
echo "  - Log file   : $LOG"
echo "  - Time stats : $TIME_LOG"


[INFO] MASKED_VCF = /mnt/qmtang/EvoFill_data/20251211_chr22/data/minor_pops_all.mask90p.vcf.gz
[INFO] REF_VCF    = /mnt/qmtang/EvoFill_data/20251211_chr22/data/major_pops_train.vcf.gz
[INFO] MAP        = /home/zqyin/mamba_test/Other_Model_test/chr22/GLIMPSE2/genetic_map_hg38_withX.txt.gz
[INFO] OUT_PREFIX = /home/zqyin/mamba_test/Other_Model_test/chr22/Eagle/minor_pops_all.mask90p.imputed.Eagle
[INFO] THREADS    = 8
[INFO] CHR        = chr22
[INFO] Indexing input VCFs if needed...
[INFO] Converting VCF to BCF...


[W::bcf_hdr_check_sanity] AC should be declared as Number=A
[W::bcf_hdr_check_sanity] AC should be declared as Number=A
[W::bcf_hdr_check_sanity] AC should be declared as Number=A
[W::bcf_hdr_check_sanity] AC should be declared as Number=A


[INFO] Running Eagle...
[INFO] Start Eagle: Mon Jan 12 10:12:31 AM CST 2026


+ /usr/bin/time -v eagle --vcfTarget=/home/zqyin/mamba_test/Other_Model_test/chr22/Eagle/tmp/target.chr22.bcf --vcfRef=/home/zqyin/mamba_test/Other_Model_test/chr22/Eagle/tmp/ref.chr22.bcf --geneticMapFile=/home/zqyin/mamba_test/Other_Model_test/chr22/GLIMPSE2/genetic_map_hg38_withX.txt.gz --chrom=chr22 --outPrefix=/home/zqyin/mamba_test/Other_Model_test/chr22/Eagle/minor_pops_all.mask90p.imputed.Eagle --vcfOutFormat=z --numThreads=8 --outputUnphased --allowRefAltSwap
+ tee /home/zqyin/mamba_test/Other_Model_test/chr22/Eagle/logs/eagle.chr22.20260112-101231.log


                      +-----------------------------+
                      |                             |
                      |   Eagle v2.4.1              |
                      |   November 18, 2018         |
                      |   Po-Ru Loh                 |
                      |                             |
                      +-----------------------------+

Copyright (C) 2015-2018 Harvard University.
Distributed under the GNU GPLv3+ open source license.

Command line options:

eagle \
    --vcfTarget=/home/zqyin/mamba_test/Other_Model_test/chr22/Eagle/tmp/target.chr22.bcf \
    --vcfRef=/home/zqyin/mamba_test/Other_Model_test/chr22/Eagle/tmp/ref.chr22.bcf \
    --geneticMapFile=/home/zqyin/mamba_test/Other_Model_test/chr22/GLIMPSE2/genetic_map_hg38_withX.txt.gz \
    --chrom=chr22 \
    --outPrefix=/home/zqyin/mamba_test/Other_Model_test/chr22/Eagle/minor_pops_all.mask90p.imputed.Eagle \
    --vcfOutFormat=z \
    --numThreads=8 \
    --outputUnphased \
    --allowRe

...

[INFO] End Eagle: Mon Jan 12 10:14:17 AM CST 2026
[INFO] Eagle timing statistics saved to: /home/zqyin/mamba_test/Other_Model_test/chr22/Eagle/logs/eagle.chr22.20260112-101231.time
[INFO] Output VCF: /home/zqyin/mamba_test/Other_Model_test/chr22/Eagle/minor_pops_all.mask90p.imputed.Eagle.vcf.gz
[INFO] QC: samples and sites
  #samples = 

[W::bcf_hdr_check_sanity] AC should be declared as Number=A


93
  #sites   = 

[W::bcf_hdr_check_sanity] AC should be declared as Number=A


190184
[DONE] Eagle finished.
  - Log file   : /home/zqyin/mamba_test/Other_Model_test/chr22/Eagle/logs/eagle.chr22.20260112-101231.log
  - Time stats : /home/zqyin/mamba_test/Other_Model_test/chr22/Eagle/logs/eagle.chr22.20260112-101231.time


### Shapeit

In [2]:
%%bash
#!/usr/bin/env bash

# === Safety options ===
set -e
set -u
set -o pipefail

# ====== Path settings (chr22) ======
MASKED_VCF="/mnt/qmtang/EvoFill_data/20251211_chr22/data/minor_pops_all.mask90p.vcf.gz"   # Target VCF with missing genotypes
REF_VCF="/mnt/qmtang/EvoFill_data/20251211_chr22/data/major_pops_train.vcf.gz"            # Reference panel (already phased)

OUT_DIR="/home/zqyin/mamba_test/Other_Model_test/chr22/Shapeit"
mkdir -p "${OUT_DIR}"

# Shapeit5 executable
SHAPEIT5_BIN="/home/zqyin/Shapeit_5.1.1/phase_common_static"

THREADS=8

echo "[INFO] Using Shapeit5: ${SHAPEIT5_BIN}"

# ====== 0. Detect chromosome name from the first variant record ======
CHR=$(bcftools view -H "${MASKED_VCF}" | head -n1 | cut -f1 || true)
if [[ -z "${CHR}" ]]; then
  echo "[ERROR] No variant records found in ${MASKED_VCF}!" >&2
  exit 1
fi
REGION="${CHR}"
echo "[INFO] REGION inferred from first variant: ${REGION}"

# ====== 1. Index original VCFs (not counted toward Shapeit time) ======
[[ -f "${MASKED_VCF}.tbi" || -f "${MASKED_VCF}.csi" ]] || bcftools index -f "${MASKED_VCF}"
[[ -f "${REF_VCF}.tbi"    || -f "${REF_VCF}.csi"    ]] || bcftools index -f "${REF_VCF}"

# ====== 2. Convert to BCF and fill tags (not counted toward Shapeit time) ======
echo "[STEP] Convert TARGET to BCF and fill tags..."
bcftools view -Ob -o "${OUT_DIR}/minor_pops_all.target.raw.bcf" "${MASKED_VCF}"
bcftools index -f "${OUT_DIR}/minor_pops_all.target.raw.bcf"

bcftools +fill-tags "${OUT_DIR}/minor_pops_all.target.raw.bcf" -- -t AC,AN,AF \
  | bcftools view -Ob -o "${OUT_DIR}/minor_pops_all.target.tags.bcf"
bcftools index -f "${OUT_DIR}/minor_pops_all.target.tags.bcf"

echo "[STEP] Convert REF to BCF and fill tags..."
bcftools view -Ob -o "${OUT_DIR}/chr22.ref.raw.bcf" "${REF_VCF}"
bcftools index -f "${OUT_DIR}/chr22.ref.raw.bcf"

bcftools +fill-tags "${OUT_DIR}/chr22.ref.raw.bcf" -- -t AC,AN,AF \
  | bcftools view -Ob -o "${OUT_DIR}/chr22.ref.tags.bcf"
bcftools index -f "${OUT_DIR}/chr22.ref.tags.bcf"

INPUT_BCF="${OUT_DIR}/minor_pops_all.target.tags.bcf"
REF_BCF="${OUT_DIR}/chr22.ref.tags.bcf"

# ====== 3. Run Shapeit5 (only this step is timed) ======
PHASED_BCF="${OUT_DIR}/minor_pops_all.mask90p.phased.shapeit5.bcf"
LOG_FILE="${OUT_DIR}/minor_pops_all.mask90p.phased.shapeit5.log"
TIME_LOG="${OUT_DIR}/minor_pops_all.mask90p.phased.shapeit5.time.log"

echo "[STEP] Run Shapeit5 phase_common (no genetic map, default 1 cM/Mb)..."
echo "[INFO] Start Shapeit5: $(date)"

/usr/bin/time -v \
  "${SHAPEIT5_BIN}" \
    --input     "${INPUT_BCF}" \
    --reference "${REF_BCF}" \
    --region    "${REGION}" \
    --output    "${PHASED_BCF}" \
    --thread    "${THREADS}" \
    --log       "${LOG_FILE}" \
  2> "${TIME_LOG}"

echo "[INFO] End Shapeit5: $(date)"
echo "[INFO] Shapeit5 timing statistics saved to: ${TIME_LOG}"

bcftools index -f "${PHASED_BCF}"

echo "[OK] Shapeit5 completed (minor_pops_all):"
echo "  - Phased BCF : ${PHASED_BCF}"
echo "  - Log        : ${LOG_FILE}"
echo "  - Time log   : ${TIME_LOG}"

# ====== 3.5 Convert phased BCF to VCF.GZ and index (not counted toward Shapeit time) ======
PHASED_VCFGZ="${OUT_DIR}/minor_pops_all.mask90p.phased.shapeit5.vcf.gz"

echo "[STEP] Convert phased BCF -> VCF.GZ..."
bcftools view -Oz -o "${PHASED_VCFGZ}" "${PHASED_BCF}"

echo "[STEP] Index VCF.GZ (tabix)..."
tabix -f -p vcf "${PHASED_VCFGZ}"

echo "[OK] Exported:"
echo "  - Phased VCF.GZ : ${PHASED_VCFGZ}"
echo "  - Index         : ${PHASED_VCFGZ}.tbi"

# ====== 4. Rough comparison of missing genotype rate (before vs after, not counted toward Shapeit time) ======
echo "[STEP] Roughly estimate GT missing rate (before vs after)"

echo -n "  [Before] MASKED_VCF missing rate: "
bcftools query -f '[%GT]\n' "${MASKED_VCF}" | tr '|' '/' | \
awk 'BEGIN{FS="[ /]"}{for(i=1;i<=NF;i++) if($i==".") m++; t+=NF} END{if(t>0) printf("%.4f\n", m/t); else print "NA"}'

echo -n "  [After ] PHASED_BCF  missing rate: "
bcftools query -f '[%GT]\n' "${PHASED_BCF}" | tr '|' '/' | \
awk 'BEGIN{FS="[ /]"}{for(i=1;i<=NF;i++) if($i==".") m++; t+=NF} END{if(t>0) printf("%.4f\n", m/t); else print "NA"}'

echo "[ALL DONE] Output directory: ${OUT_DIR}"


[INFO] Using Shapeit5: /home/zqyin/Shapeit_5.1.1/phase_common_static


[W::bcf_hdr_check_sanity] AC should be declared as Number=A


[INFO] REGION inferred from first variant: chr22
[STEP] Convert TARGET to BCF and fill tags...


[W::bcf_hdr_check_sanity] AC should be declared as Number=A
[W::bcf_hdr_check_sanity] AC should be declared as Number=A
[W::bcf_hdr_check_sanity] AC should be declared as Number=A
[W::bcf_hdr_check_sanity] AC should be declared as Number=A
[W::bcf_hdr_check_sanity] AC should be declared as Number=A


[STEP] Convert REF to BCF and fill tags...


[W::bcf_hdr_check_sanity] AC should be declared as Number=A
[W::bcf_hdr_check_sanity] AC should be declared as Number=A
[W::bcf_hdr_check_sanity] AC should be declared as Number=A
[W::bcf_hdr_check_sanity] AC should be declared as Number=A
[W::bcf_hdr_check_sanity] AC should be declared as Number=A


[STEP] Run Shapeit5 phase_common (no genetic map, default 1 cM/Mb)...
[INFO] Start Shapeit5: Mon Jan 12 10:21:03 AM CST 2026

[SHAPEIT5] phase_common (jointly phase multiple common markers)
  * Author        : Olivier DELANEAU, University of Lausanne
  * Contact       : olivier.delaneau@gmail.com
  * Version       : 5.1.1 / commit = 990ed0d / release = 2023-05-08
  * Run date      : 12/01/2026 - 10:21:03

Files:
  * Input         : [/home/zqyin/mamba_test/Other_Model_test/chr22/Shapeit/minor_pops_all.target.tags.bcf]
  * Reference     : [/home/zqyin/mamba_test/Other_Model_test/chr22/Shapeit/chr22.ref.tags.bcf]
  * Output        : [/home/zqyin/mamba_test/Other_Model_test/chr22/Shapeit/minor_pops_all.mask90p.phased.shapeit5.bcf]
  * Output format : [bcf]
  * Output LOG    : [/home/zqyin/mamba_test/Other_Model_test/chr22/Shapeit/minor_pops_all.mask90p.phased.shapeit5.log]

Parameters:
  * Seed    : 15052011
  * Threads : 8 threads
  * MCMC    : 15 iterations [5b + 1p + 1b + 1p + 1b + 1p +

[W::bcf_hdr_check_sanity] AC should be declared as Number=A


0.0191
  [After ] PHASED_BCF  missing rate: 0.0000
[ALL DONE] Output directory: /home/zqyin/mamba_test/Other_Model_test/chr22/Shapeit
